# Desafio Data Science Intelivix
  Por: [Paulo Romeira](https://github.com/pauloromeira)

In [1]:
# Classificadores
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# Pré-processamento de dados
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

# Treinamento
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, GridSearchCV

# Métricas
from sklearn.metrics import classification as class_metrics

# Outros
import numpy as np
import pandas as pd
from itertools import count
from IPython.display import display

# Dataset
# from sklearn.datasets import load_iris

# Random state: seed para que o notebook possa ser reproduzido.
R_STATE = 1

# Dataset

In [2]:
# Easy way :)
# iris = load_iris()
# X, y = iris.data, iris.target

header_names = ['sepal length', 'sepal width', 'petal length', 'petal width', 'class']
ds = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data',
                 header=None,
                 names=header_names
                )

display(ds.groupby('class').first())

,sepal length,sepal width,petal length,petal width
class,,,,
Iris-setosa,5.1,3.5,1.4,0.2
Iris-versicolor,7.0,3.2,4.7,1.4
Iris-virginica,6.3,3.3,6.0,2.5


## Pré-processamento dos dados

In [3]:
# Mapeia as classes para números
encoder = LabelEncoder()

# Normaliza os atributos para valores entre 0 e 1
scaler = MinMaxScaler()

# Apenas as variáveis X e y serão usadas daqui em diante
X = scaler.fit_transform(ds[header_names[:-1]])
y = encoder.fit_transform(ds['class'])

# Classificadores e intervalos de hiperparâmetros
* Obs: é possível acrescentar novos classificadores e parâmetros nesta seção, sem a necessidade de alterar o resto do código.

In [4]:
classifiers = [
    KNeighborsClassifier(),
    SVC(),
    LogisticRegression()
]

classifiers_names = ['KNN', 'SVC', 'L. Regression']

# Definindo os valores possíveis dos hiperparâmetros dos classificadores
params = [
    { 'n_neighbors': range(1, 50), 'weights': ['uniform', 'distance'] }, # KNN
    { 'kernel':('linear', 'rbf'), 'C': np.linspace(0.001, 5, 500) }, # SVC
    { 'C': range(1, 500) } # L. Regression
]

## Otimização de hiperparâmetros

In [5]:
# Os dados são divididos em 3 (33% para validação) e cada parâmetro será testado 3 vezes - um para cada "fold".
# O StratifiedKFold é usado para dividir de forma balanceada as classes das amostras.
cross_validator = StratifiedKFold(n_splits=3, shuffle=True, random_state=R_STATE)

for classifier, name, param in zip(classifiers, classifiers_names, params):
    # Aqui, cada classificador é testado em cross-validation pelo GridSearch,
    # para cada combinação possível de parâmetros.
    grid = GridSearchCV(classifier, param, scoring='accuracy', cv=cross_validator).fit(X, y)

    # Setando parâmetros achados no gridsearch
    classifier.set_params(**grid.best_params_)
    
    print('{0}: {1} - {2:.2%}'.format(name, grid.best_params_, grid.best_score_))

KNN: {'n_neighbors': 4, 'weights': 'uniform'} - 96.67%
SVC: {'C': 2.3652565130260519, 'kernel': 'rbf'} - 97.33%
L. Regression: {'C': 58} - 94.67%


> Na busca pelos melhores parâmetros acima, o classificador que obteve melhor desempenho no cross-validation foi o _SVC_, com 97.33% na média de acertos.

## Treinamento dos classificadores

Com os hiperparâmetros definidos, os classificadores são treinados e testados massivamente, com diferentes combinações da amostra.

In [6]:
# Esta função divide os dados randomicamente, na proporção definida por
# n_splits (3 == 2/3, ou 66% em treino, 1/3 em teste), treina os classificadores,
# e retorna uma lista com os dados reais de teste, e uma lista de listas com os 
# valores inferidos pelos classificadores.
def massive_training(classifiers, X, y, n_iterations, n_splits=3):
    y_true = []
    y_preds = [[] for _ in classifiers]

    for i in range(n_iterations):
        cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=(R_STATE + i))
        for i_train, i_test in cv.split(X, y):
            X_train, X_test = X[i_train], X[i_test]
            y_train, y_test = y[i_train], y[i_test]
            
            y_true.extend(y_test)
            for j, classifier in enumerate(classifiers):
                classifier.fit(X_train, y_train)
                y_preds[j].extend(classifier.predict(X_test))
    
    return y_true, y_preds

y_true, y_preds = massive_training(classifiers, X, y, 1000)

# Análise dos classificadores

In [7]:
for c_name, y_pred in zip(classifiers_names, y_preds):
    print('*** {0}: {1:.2%} ***'.
          format(c_name, class_metrics.accuracy_score(y_true, y_pred)))
    print('\nConfusion Matrix:')
    print(class_metrics.confusion_matrix(y_true, y_pred))
    print('\nReport:')
    print(class_metrics
          .classification_report(y_true, y_pred, target_names=encoder.classes_, digits=4))
    print()

*** KNN: 95.87% ***

Confusion Matrix:
[[50000     0     0]
 [    0 48480  1520]
 [    0  4672 45328]]

Report:
                 precision    recall  f1-score   support

    Iris-setosa     1.0000    1.0000    1.0000     50000
Iris-versicolor     0.9121    0.9696    0.9400     50000
 Iris-virginica     0.9676    0.9066    0.9361     50000

    avg / total     0.9599    0.9587    0.9587    150000


*** SVC: 96.18% ***

Confusion Matrix:
[[50000     0     0]
 [    0 48657  1343]
 [    0  4394 45606]]

Report:
                 precision    recall  f1-score   support

    Iris-setosa     1.0000    1.0000    1.0000     50000
Iris-versicolor     0.9172    0.9731    0.9443     50000
 Iris-virginica     0.9714    0.9121    0.9408     50000

    avg / total     0.9629    0.9618    0.9617    150000


*** L. Regression: 94.31% ***

Confusion Matrix:
[[49965    35     0]
 [    0 45397  4603]
 [    0  3891 46109]]

Report:
                 precision    recall  f1-score   support

    Iris-setosa   